In [1]:
GLOBAL_LOADED_PYMATH=True

In [53]:
import numpy as np
import matplotlib.pyplot as plt
from functools import cmp_to_key
from random import shuffle
import abc

In [185]:
def binarySearch(data, val,cmp=lambda x,y: -1 if x<y else (1 if x>y else 0) ): #If no exact match is found gives closest LOWER index (i.e. 2 for both 2.8 and 2.2)
    lo, hi = 0, len(data) - 1
    best_ind = lo
    #print(data,val)
    while lo <= hi:
        mid = lo + (hi - lo) // 2
        cmpv=cmp(data[mid],val)
        if cmpv==-1:
            lo = mid + 1
        elif cmpv==1:
            hi = mid - 1
        else:
            best_ind = mid
            return best_ind
    return (lo,hi) if cmp(data[lo],data[hi])<0 else (hi,lo)
    #best_ind=lo if cmp(data[lo],data[hi])<0 else hi
    #return best_ind

In [346]:
class OutOfBoundsError(BaseException): pass
def linearInterpolation(xl,xu,xt,y1,y2):
    if xl==xu: return None
    d1=abs(xt-xl)
    d2=abs(xu-xt)
    return (d1*y2+d2*y1)/(d1+d2)
CMPSTART=0
def tupleSortCmp(x,y,startIndex=CMPSTART):
    for i in range(startIndex,len(x)):
        if x[i]==y[i]: continue
        else: return -1 if x[i]<y[i] else 1
    for i in range(0,startIndex):
        if x[i]==y[i]: continue
        else: return -1 if x[i]<y[i] else 1
    return 0
tupleSortKey=cmp_to_key(tupleSortCmp)
def samePoint(p1,p2,threshold):
    return np.all(np.abs(p1-p2)<threshold)

class InterpolatedFunction:
    def __init__(self,x,y,interpolation=linearInterpolation,prec=1e-3): #x,y can be lists/tuples for multidimensional functions
        x=np.array(x)
        y=np.array(y)
        try:
            dim=len(x[0])
        except:
            dim=1
        x=x.reshape(len(x),-1)
        y=y.reshape(len(y),1) #If this fails, data is bad
        xlst=list(x)
        srt=np.argsort([tupleSortKey(v) for v in xlst])
        x=x[srt]
        y=y[srt]
        self.x=x
        self.y=y
        self.dim=dim
        self.prec=prec
        self.dimx=[x]
        self.dimy=[y]
        self.interpol=interpolation
        ldimlims,udimlims=[np.min(x[:,0])],[np.max(x[:,0])]
        for i in range(1,self.dim):
            CMPSTART=i
            mykey=cmp_to_key(tupleSortCmp)
            srt=np.argsort([mykey(v) for v in xlst])
            x=x[srt]
            y=y[srt]
            self.dimx.append(x.copy())
            self.dimy.append(y.copy())
            ldimlims.append(np.min(x[:,i]))
            udimlims.append(np.max(x[:,i]))
        self.ldimlims=np.array(ldimlims)
        self.udimlims=np.array(udimlims)
    
    def getValueAtInternal(self,xv,usedim=0):
        use=self.dimx[usedim]
        usey=self.dimy[usedim]
        idx=binarySearch(list(use),xv,cmp=tupleSortCmp)
        if type(idx)==int: return self.x[idx],self.y[idx],True
        else: return (self.x[idx[0]],self.x[idx[1]]),(usey[idx[0]],usey[idx[1]]),False
    def getValueAt(self,xv):
        if type(xv)==int or type(xv)==float: xv=[xv]
        if not (np.all(xv<self.udimlims) and np.all(xv>self.ldimlims)): raise OutOfBoundsError()
        svs=[]
        for i in range(self.dim):
            p,v,s=self.getValueAtInternal(xv,usedim=i)
            if s: return float(v)
            #print("At dim =",i,v,"appended with closest point",p)
            svs.append(self.interpol(p[0][i],p[1][i],xv[i],v[0],v[1]))
        return np.mean([v for v in svs if v is not None])
    def __call__(self,xv): return self.getValueAt(xv)

In [347]:
def make1Dpotential(steadiness=0.8,stepX=0.01,stepY=0.05,scaleX=(-0.5,0.5),scaleY=(0,1),noise=0.8,smoothen=2,symmetric=False,periodic=False): #Noise is a fraction of step
    smoothen*=2
    noise*=stepY
    x=0
    y=np.random.uniform(0,1)
    xs=[]
    ys=[]
    dirx=1
    while x<1.0:
        xs.append(x)
        ys.append(y)
        if np.random.uniform(0,1)<(1-steadiness): dirx*=-1
        x+=stepX
        y+=np.abs(np.random.normal(stepY,noise))*dirx
    ys=[ys[0] for _ in range(smoothen//2)]+ys+[ys[0] for _ in range(smoothen//2)]
    xs=np.array(xs)
    ys=np.array(ys)
    for _ in range(smoothen): ys=(ys[1:]+ys[:-1])/2
    if symmetric: ys[:len(ys)//2][:len(ys)//2]=ys[:len(ys)//2-1:-1][:len(ys)//2]
    if periodic and not symmetric:
        #ys=np.concatenate((ys[len(ys)//2:]
        midval=ys[len(ys)//2]
        ys[:len(ys)//2]/=ys[0]
        ys[len(ys)//2:]/=ys[-1]
    xs=(xs+scaleX[0])*(scaleX[1]-scaleX[0])
    r=(np.max(ys)-np.min(ys))/(scaleY[1]-scaleY[0])
    ys/=r
    return xs,ys